<a href="https://www.kaggle.com/code/raghavrastogi75/classifying-cifar-using-pytorch-beginner?scriptVersionId=91175880" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importing the libraries

In [1]:
import torchvision
import torch
import numpy as np
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import time
import pandas as pd
import torchvision.datasets as datasets
from IPython import display
import matplotlib.pyplot as plt
import torchvision.transforms

In [2]:
batch_size = 64
transform = transforms.Compose([transforms.Resize(64), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_ds = DataLoader(datasets.CIFAR10("data", train=True, transform=transform, download=True), batch_size=batch_size)
test_ds = DataLoader(datasets.CIFAR10("data", train=False, transform=transform, download=True), batch_size=batch_size)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [3]:
class CNNModel(nn.Module):
    
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 6 * 6, 8192),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(8192, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
model = CNNModel(num_classes=10)

In [5]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

model.to(device)

CNNModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU()
    (13): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=18432, out_fe

In [7]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [8]:
train(model, optimizer,torch.nn.CrossEntropyLoss(), train_ds,test_ds, epochs=20, device=device)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0, Training Loss: 1.69, Validation Loss: 1.35, accuracy = 0.50
Epoch: 1, Training Loss: 1.29, Validation Loss: 1.14, accuracy = 0.59
Epoch: 2, Training Loss: 1.12, Validation Loss: 1.06, accuracy = 0.63
Epoch: 3, Training Loss: 1.00, Validation Loss: 1.10, accuracy = 0.63
Epoch: 4, Training Loss: 0.93, Validation Loss: 0.94, accuracy = 0.68
Epoch: 5, Training Loss: 0.86, Validation Loss: 0.88, accuracy = 0.70
Epoch: 6, Training Loss: 0.81, Validation Loss: 0.83, accuracy = 0.71
Epoch: 7, Training Loss: 0.76, Validation Loss: 0.83, accuracy = 0.72
Epoch: 8, Training Loss: 0.72, Validation Loss: 0.83, accuracy = 0.72
Epoch: 9, Training Loss: 0.69, Validation Loss: 0.80, accuracy = 0.73
Epoch: 10, Training Loss: 0.67, Validation Loss: 0.81, accuracy = 0.72
Epoch: 11, Training Loss: 0.64, Validation Loss: 0.83, accuracy = 0.73
Epoch: 12, Training Loss: 0.61, Validation Loss: 0.79, accuracy = 0.74
Epoch: 13, Training Loss: 0.58, Validation Loss: 0.83, accuracy = 0.73
Epoch: 14, Train

In [9]:
labels = np.array(train_ds.dataset.classes)
prediction_l = []
target_l = []
for batch in test_ds:
    images, targets = batch
    images = images.to(device)
    targets = targets.to(device)
    
      
    prediction = model(images).argmax(dim=1)
    print(targets)
    print(prediction)
    break
  

tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3], device='cuda:0')
tensor([5, 8, 8, 0, 6, 6, 9, 6, 3, 9, 5, 9, 5, 7, 9, 2, 5, 7, 8, 6, 7, 0, 0, 9,
        4, 5, 4, 0, 9, 6, 6, 5, 4, 5, 9, 8, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 3,
        7, 6, 9, 8, 7, 3, 8, 8, 7, 5, 4, 5, 7, 5, 6, 0], device='cuda:0')
